In [1]:
import pandas as pd
import os
import re

In [4]:
QCHAT_path = r"/home/skbae/Documents/skbae/ASD/paper/Pgm/Final/Submitted/input/qchat.csv"

# SCQ sheet - Column D (En questions) , Make Dataframe
QCHAT_en = pd.read_csv(QCHAT_path)
QCHAT_en

,Var,Q_chat_10_T
0,A1,Does your child look at you when you call his...
1,A2,How easy is it for you to get eye contact with...
2,A3,Does your child point to indicate that s/he wa...
3,A4,Does your child point to share interest with y...
4,A5,"Does your child pretend? (e.g. care for dolls,..."
5,A6,Does your child follow where you’re looking?
6,A7,If you or someone else in the family is visibl...
7,A8,Would you describe your child’s first words as:
8,A9,Does your child use simple gestures? (e.g. wav...
9,A10,Does your child stare at nothing with no appar...


Sentence Transformer

In [6]:
import torch
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [7]:
# Check if CUDA is available and set the device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'

data2 = pd.read_csv(r"/home/skbae/Documents/skbae/ASD/paper/Pgm/Final/Submitted/input/cdb_advanced_ASD_F3.csv")

In [10]:
# 2. Text Preprocessing
def preprocess_text(text):
    return " ".join([word.lower() for word in text.split()])
data2['Processed_Name'] = data2['Name'].apply(preprocess_text)

# 3. Vectorization using Sentence Transformers
model = SentenceTransformer('all-MiniLM-L6-v2')  # replace with your model
model = model.to(device)  # Move the model to the device
X_full = model.encode(data2['Processed_Name'].tolist(), convert_to_tensor=True)
X_full = X_full.to(device)  # Move X_full to the device

# 4. KMeans clustering with 100 clusters (assuming it was done earlier)
kmeans_100 = KMeans(n_clusters=100, random_state=42).fit(X_full.cpu())  # KMeans requires CPU tensor
data2['100_Cluster'] = kmeans_100.predict(X_full.cpu())  # KMeans requires CPU tensor

# Function to find best matches in dataset based on the processed text and include rank
def match_question_to_data_detailed(question, data2, model, top_n=None):
    question_vec = model.encode([preprocess_text(question)], convert_to_tensor=True)
    question_vec = question_vec.to(device)  # Move tensor to the device
    cosine_similarities = cosine_similarity(question_vec.cpu(), X_full.cpu()).flatten()  # Compute cosine similarity on CPU
    if top_n:
        related_docs_indices = cosine_similarities.argsort()[-top_n:][::-1]
    else:
        related_docs_indices = cosine_similarities.argsort()[::-1]
    matched_data = data2.iloc[related_docs_indices]
    matched_data['Question']= question
    matched_data['Matching Rank/Probability'] = cosine_similarities[related_docs_indices]
    return matched_data


# Find the best matches for each question with detailed information
question_matches_detailed_ranked = {}
for idx, question in enumerate(QCHAT_en['Q_chat_10_T']):  # Change SCQ_en to QCHAT_en
    matched_data = match_question_to_data_detailed(question, data2, model)
    question_matches_detailed_ranked[f"Q_chat_10_T {idx}"] = matched_data[['ID','Question', 'Name', 'CUI','DSM5 Criteria','Symptom of DSM5 Criteria', 'Matching Rank/Probability', '100_Cluster']]

/home/skbae/anaconda3/envs/myNLP/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/tmp/ipykernel_1565998/1436535079.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_data['Question']= question
/tmp/ipykernel_1565998/1436535079.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_data['Matching Rank/Probability'] = cosin

In [11]:
# 8. Save the results to an Excel file

output_filepath_questions_detailed_ranked = r"/home/skbae/Documents/skbae/ASD/paper/Pgm/Final/Submitted/qchat_en_st.xlsx"
writer = pd.ExcelWriter(output_filepath_questions_detailed_ranked, engine='xlsxwriter')
for q_key, matched_data in question_matches_detailed_ranked.items():
    # Add the question as a separate row before the matches
    dataframe = pd.DataFrame([[q_key, None, None, None, None, None, None, None]], columns=['Question', 'ID', 'Name', 'CUI', 'DSM5 Criteria', 'Symptom of DSM5 Criteria', 'Matching Rank/Probability', '100_Cluster'])
    dataframe = pd.concat([dataframe, matched_data], ignore_index=True)
    dataframe.to_excel(writer, sheet_name=q_key, index=False)

# Use close() instead of save()
writer.close()